In [9]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
from urllib.parse import quote
from logging import exception
from tqdm import tqdm
import pandas as pd
import numpy as np
import platform
import time
import click
import sys
import os
import re

#### 로그인

In [10]:
driver = Chrome("c:/pydata/chromedriver.exe")
url =" https://www.instagram.com/"
# uid = "lapras_1018"
# upw = "00tlsghk"
uid = "qlenfqlenfrl"
upw = "0!cndwntlsghk"
driver.get(url)
time.sleep(2)
driver.find_element(By.NAME, "username").send_keys(uid)
driver.find_element(By.NAME, "password").send_keys(upw)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()

time.sleep(4)


C:\Users\CJ\AppData\Local\Temp\ipykernel_15632\3021933637.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("c:/pydata/chromedriver.exe")


#### 크롤링 파트

In [11]:
cloth = pd.read_csv("C:/Users/CJ/Desktop/MultiCampus/프로젝트_3_마지막/크롤링/csv/옷스타그램_id_url_src_0707.csv")
content_lst =[]
reply_lst = []
like_lst =[]
for j in cloth.index:
    num =  j
    im_url = cloth.iloc[num:,-2]
    im_url = "https://www.instagram.com/" + im_url.values[0]
    driver.get(im_url)
    time.sleep(3)
    
    # 게시글 크롤링 및 더보기 버튼 유무 확인
    try:    
        html = driver.page_source
        soup = bs(html, "html.parser")
        content = soup.find("div", class_="_a9zs").getText()
        img_name = cloth.iloc[num,0]
        content_lst.append({img_name:content})
        # 좋아요 가져오기
        if soup.find("div","_aacl _aaco _aacw _aacx _aada _aade").get_text() == "여러 명":
            like_lst.append(-1)
        else :
            like_lst.append(soup.find("div","_aacl _aaco _aacw _aacx _aada _aade").find("span").get_text())
        # 더 보기 버튼 몇개 있는지 확인
        button_lst = soup.find("ul",class_="_a9z6 _a9za").find_all('button', class_="_acan _acao _acas")
        text = soup.find("ul", class_="_a9z6 _a9za").find_all("li")[-1].get_text()
        plus_button = soup.find("ul", class_="_a9z6 _a9za").find_all("li")[-1].find_all("button")
        if len(plus_button[0].find_all("svg")) == 0: plus_button = []
        elif plus_button[0].find_all("svg") != '댓글 더 읽어들이기' : plus_button = []
    except Exception as e:
        print(str(num)+ " : url오류")

        
    # 댓글 더 보기 확인
    cnt = 0 # 4번 이상 더 누르면 그만
    while len(plus_button) != 0 :
        if cnt > 4 : break
        if plus_button == 0 : break;
        driver.find_element(By.CSS_SELECTOR, 'div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p > div > div._aasx > div._aat6 > ul > li > div > button').send_keys(Keys.ENTER)
        time.sleep(2)
        html = driver.page_source
        soup = bs(html, "html.parser")
        plus_button = soup.find("ul", class_="_a9z6 _a9za").find_all("li")[-1].find_all("button")
        if len(plus_button[0].find_all("svg")) == 0: break
        elif plus_button[0].find_all("svg") != '댓글 더 읽어들이기' : break
        cnt += 1
    time.sleep(1)
    
    # 답글 더 보기 확인
    while len(button_lst) != 0:
        start = 2; end = len(button_lst)+2
        for i in range(start, end):
            if len(button_lst) > 1:
                n = str(i)
                try:
                    driver.find_element(By.CSS_SELECTOR, f'div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p > div > div._aasx > div._aat6 > ul > ul:nth-child({n}) > li > ul > li > div > button').send_keys(Keys.ENTER)
                except:
                    print("error")
            else:
                driver.find_element(By.CSS_SELECTOR, f'div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p > div > div._aasx > div._aat6 > ul > ul > li > ul > li > div > button').send_keys(Keys.ENTER)
        html = driver.page_source
        soup = bs(html, "html.parser")
        buttons = soup.find("ul",class_="_a9ym").find_all('button', class_="_acan _acao _acas")
        button_lst = [button for button in buttons if "답글 보기" in button.get_text()]
    time.sleep(1)

    # 댓글 크롤링
    html = driver.page_source
    soup = bs(html, "html.parser")
    ul_lst = soup.find_all("ul", "_a9ym")
    ul_lsts = []
    for ul in ul_lst:
        for span in ul.find_all("span", "_aacl _aaco _aacu _aacx _aad7 _aade"):
            ul_lsts.append(span.get_text())
    reply_lst.append({img_name:ul_lsts})

error
error


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p > div > div._aasx > div._aat6 > ul > ul > li > ul > li > div > button"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F86463+2188387]
	Ordinal0 [0x00F1E461+1762401]
	Ordinal0 [0x00E33D78+802168]
	Ordinal0 [0x00E61880+989312]
	Ordinal0 [0x00E61B1B+989979]
	Ordinal0 [0x00E8E912+1173778]
	Ordinal0 [0x00E7C824+1099812]
	Ordinal0 [0x00E8CC22+1166370]
	Ordinal0 [0x00E7C5F6+1099254]
	Ordinal0 [0x00E56BE0+945120]
	Ordinal0 [0x00E57AD6+948950]
	GetHandleVerifier [0x012271F2+2712546]
	GetHandleVerifier [0x0121886D+2652765]
	GetHandleVerifier [0x0101002A+520730]
	GetHandleVerifier [0x0100EE06+516086]
	Ordinal0 [0x00F2468B+1787531]
	

'여러 명'

In [340]:
reply_lst

[{'6007781.jpg': ['바지 되게 핏도좋고 시원해보이네요 🤤🤤', 'Collab? Dm @yubamajewels', '👑']},
 {'6007782.jpg': ['#2022ss_여름도씨',
   '세트도 너무 예쁜데, 단품으로 코디해도 이쁠꺼같아서 무조건 단품이여!!!',
   '요런옷은 세트가  이쁠거 같아요❤️ 완성도도 높구요😊😊 뭐 입을지 고르는 고민도 없을 것 같아요',
   '치마가 짧아서 ㅠㅠ 단품이요 😢',
   '상의가 안어울리는 체형이라서 ㅜㅜㅜㅜㅜ 개별구매요!!!!!',
   '세트요!!!!!!!!😍',
   '전 단품이요!! 치마가 너무 짧아서 아마 셋업은 어려울꺼같어여,,,🥲',
   '요건 셋업으로 입어줘야 더 예쁠 것 같아요 🙌🏻 셋업이기 때문에 나올 수 있는 분위기가 매력을 업그레이드 해주는 느낌...? 도씨.. 자꾸 지갑 열리게 하네... 🤑',
   '단품이용',
   '단품이용🙌',
   '세트로 입고 싶을 것 같아요😍',
   '헐 무조건 세트요 저 세트에 레인부츠 신을 생각에 비 와도 너무 신날듯🔥🔥🔥',
   '이건 세트가 찰떡일것같아요..😍',
   '세트요❤️',
   '세트요!!!!!!😍',
   '단품이욧 ,,저렇게 짧은 치마는 초큼 어려워요,,',
   '단품이요!!!!!❤️',
   '세트욤🔥',
   '세트로 입기에 치마길이가 좀 짧은거 같아요ㅜㅜ 단품!!구매가 나을거 같아요 ㅎㅎ',
   '단품이요!! 블라우스 너무 탐나요..😢']},
 {'6007783.jpg': ['여녜인',
   '예',
   '울 공주 실물도 못 담는 하루필름녀석 ,,',
   '노력은 해드릴게',
   '이쁘게 안볼수가업짜나 ㅜ',
   '☁️']},
 {'6007784.jpg': ['😍😍😍😍😍😍😍😍', '😍😍😍😍😍😍', '❤️']},
 {'6007785.jpg': ['nice😍😍😂😂😂',
   '😍😍😍',
   '이쁘네^^❤️',
   '여기가 좋반 맛집이라면서요!!??ㅋㅋ',
   '공주얌 💙',
   '바디 크리에이터